In [1]:
import sys
print("python 版本检测",sys.version)

python 版本检测 3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]


In [2]:
#gpu检测
import torch
print(f"PyTorch version: {torch.__version__}")
!nvidia-smi
print("torch可使用的GPU数量是",torch.cuda.device_count())

PyTorch version: 2.0.0+cu117
Fri Jun 14 18:07:51 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:09.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                           

In [3]:
# %pip install psutil gputil ipywidgets IPython

In [4]:
import psutil
import GPUtil
import ipywidgets as widgets
from IPython.display import display
import time
import threading

# 监控线程的终止标志
stop_thread = threading.Event()

cpu_label = widgets.Label()
gpu_label = widgets.Label()
net_label = widgets.Label()
disk_label = widgets.Label()

def update_metrics():
    while not stop_thread.is_set():
        # Update CPU usage
        cpu_usage = psutil.cpu_percent(interval=1)
        cpu_label.value = f"CPU 使用率: {cpu_usage}%"

        # Update GPU usage
        gpus = GPUtil.getGPUs()
        gpu_info = ""
        for gpu in gpus:
            gpu_info += (f"GPU {gpu.id}: GPU使用率: {gpu.load * 100}%, "
                         f"显存总量: {gpu.memoryTotal}MB, "
                         f"已用显存: {gpu.memoryUsed}MB, "
                         f"可用显存: {gpu.memoryFree}MB\n")
        gpu_label.value = gpu_info

        # Update memory usage
        memory = psutil.virtual_memory()
        memory_total = memory.total / (1024 * 1024 * 1024)  # Convert to GB
        memory_used = memory.used / (1024 * 1024 * 1024)  # Convert to GB
        memory_label = f"内存: {memory_used:.2f}GB / {memory_total:.2f}GB"
        cpu_label.value += f"\n{memory_label}"

        # Update disk usage
        disk_usage = psutil.disk_usage('/')
        disk_label.value = (f"磁盘使用率: {disk_usage.percent}% "
                            f"({disk_usage.used / 1024 / 1024 / 1024:.2f} GB 使用中 / "
                            f"{disk_usage.total / 1024 / 1024 / 1024:.2f} GB 总共)")

        # Update network usage
        net_io = psutil.net_io_counters()
        net_sent_speed = net_io.bytes_sent - getattr(update_metrics, 'last_net_sent', 0)
        net_recv_speed = net_io.bytes_recv - getattr(update_metrics, 'last_net_recv', 0)
        update_metrics.last_net_sent = net_io.bytes_sent
        update_metrics.last_net_recv = net_io.bytes_recv
        net_label.value = f"网络上传速度: {net_sent_speed / 1024 / 1024:.2f} MB/s\n网络下载速度: {net_recv_speed / 1024 / 1024:.2f} MB/s"

        # 使用较短的 sleep 间隔来检查终止标志
        for _ in range(10):
            if stop_thread.is_set():
                break
            time.sleep(0.1)

# 启动监控线程
thread = threading.Thread(target=update_metrics)
thread.start()

# 显示标签
display(cpu_label)
display(gpu_label)
display(net_label)
display(disk_label)

# 添加停止按钮
stop_button = widgets.Button(description="停止监控")
display(stop_button)

# 停止按钮的回调函数
def stop_monitoring(b):
    stop_thread.set()
    thread.join()  # 等待线程结束
    print("监控已停止")

stop_button.on_click(stop_monitoring)


Label(value='')

Label(value='')

Label(value='')

Label(value='')

Button(description='停止监控', style=ButtonStyle())

In [5]:
import jittor as jt
from jittor import nn

class LinearModel(nn.Module):
    def __init__(self):
        self.linear = nn.Linear(10, 2)
    def execute(self, x):
        x = self.linear(x)
        return x
net = LinearModel().cuda()
x = jt.random((10, 10))
out = net(x)
print(out.shape)
net


[i 0614 18:07:52.301122 00 log.cc:351] Load log_sync: 1
[i 0614 18:07:52.439783 00 compiler.py:956] Jittor(1.3.8.5) src: /root/miniconda3/envs/pyt/lib/python3.10/site-packages/jittor
[i 0614 18:07:52.442891 00 compiler.py:957] g++ at /usr/bin/g++(9.4.0)
[i 0614 18:07:52.443545 00 compiler.py:958] cache_path: /root/.cache/jittor/jt1.3.8/g++9.4.0/py3.10.11/Linux-5.4.0-16xf1/IntelRXeonRPlax40/default
[i 0614 18:07:52.446949 00 __init__.py:411] Found /usr/local/cuda/bin/nvcc(11.7.99) at /usr/local/cuda/bin/nvcc.
[i 0614 18:07:52.449912 00 __init__.py:411] Found addr2line(2.34) at /usr/bin/addr2line.
[i 0614 18:07:52.740186 00 compiler.py:1011] cuda key:cu11.7.99_sm_75
[i 0614 18:07:53.085750 00 __init__.py:227] Total mem: 30.60GB, using 10 procs for compiling.
[i 0614 18:07:53.246618 00 jit_compiler.cc:28] Load cc_path: /usr/bin/g++
[i 0614 18:07:55.411497 00 init.cc:62] Found cuda archs: [75,]
[i 0614 18:07:56.545367 00 cuda_flags.cc:49] CUDA enabled.


[10,2,]


LinearModel(
    linear: Linear(10, 2, float32[2,], None)
)

In [6]:
net = LinearModel().cuda(0)
jt.flags.use_cuda = 1

In [7]:
net.cuda(device='gpu')

LinearModel(
    linear: Linear(10, 2, float32[2,], None)
)